# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', color='City', alpha=0.6, s='Humidity', scale=1)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
clean_df = city_data_df.loc[(city_data_df["Max Temp"]>=0)\
                            &(city_data_df["Cloudiness"]<=95)\
                            &(city_data_df["Humidity"]<=65),:]

# Drop any rows with null values
clean_df.dropna()

# Display sample data
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df.iloc[:,[1, 8, 2, 3, 5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
2,new norfolk,AU,-42.7826,147.0587,58,
4,lanzhou,CN,36.0564,103.7922,48,
12,trairi,BR,-3.2778,-39.2689,51,
22,guiratinga,BR,-16.3494,-53.7583,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'filter': '',
    'bias':'',
    'apiKey':geoapify_key,
    'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
lanzhou - nearest hotel: 西园宾馆
trairi - nearest hotel: Pousada Villa Aurora
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: Chez Bianca & Benoit
constitucion - nearest hotel: Hostal Alameda
saint-louis - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
naberera - nearest hotel: No hotel found
drochia - nearest hotel: Hotel Confort
collierville - nearest hotel: Days Inn & Suites by Wyndham Collierville Germantown Area
cayenne - nearest hotel: Central Hôtel
banda - nearest hotel: #acnindiafy21
caravelas - nearest hotel: Pousada dos Navegantes
nalut - nearest hotel: No hotel found
solnechnyy - nearest hotel: Заря
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
shenjiamen - nearest hotel: Zhonghan Hotel
itarema - nearest hotel: Pousada Oásis
heihe - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora
22,guiratinga,BR,-16.3494,-53.7583,14,No hotel found
...,...,...,...,...,...,...
555,bulgan,MN,48.8125,103.5347,40,Bulgan Hotel
560,changli,CN,39.7042,119.1506,33,客房
561,choma,ZM,-16.8065,26.9531,22,No hotel found
565,wyndham,US,37.6985,-77.6125,52,Home2 Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', color='City', alpha=0.8, hover=True, hover_cols='all', s='Humidity', scale=1)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)